## Conversational Requests
Interaction with contents and systems prompts using one model or combination of models.

In [ ]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display

from openai import OpenAI

load_dotenv(override=True)

In [ ]:
# init models names
GPT_4_1_MINI = "gpt-4.1-mini"
GPT_4_1 = "gpt-4.1"
GPT_5_MINI = "gpt-5-mini"
GPT_5 = "gpt-5"

GEMINI_2_5_FLASH = "gemini-2.5-flash"
GEMINI_2_5_PRO = "gemini-2.5-pro"

GROQ_LLAMA_3_1 = "llama-3.1-8b-instant"
GROQ_LLAMA_3_3 = "`llama-3.3-70b-versatile"
GROQ_GPT_OSS_20 = "openai/gpt-oss-20b"
GROQ_GPT_OSS_120 = "openai/gpt-oss-120b"

In [ ]:
# init api keys
groq_api_key = os.getenv("GROQ_API_KEY")
gemini_api_key = os.getenv("GEMINI_API_KEY")

In [ ]:
# init clients
openai_client = OpenAI()

gemini_client = OpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

groq_client = OpenAI(
    api_key=groq_api_key,
    base_url="https://api.groq.com/openai/v1"
)

In [ ]:
def send_message(client: OpenAI, model: str, message: str, messages: list[dict], max_tokens: int = 1024) -> tuple[str|None, list[dict]]:
    """
    Send a message to the specified model using the provided OpenAI client.

    Args:
        client (OpenAI): The OpenAI client to use for the request.
        model (str): The model name to use for the request.
        message (str): The user message to send.
        messages (list[dict]): The conversation history.
        max_tokens (int): The maximum number of tokens to generate in the response.

    Returns:
        tuple[str|None, list[dict]]: The model's response and the updated conversation history.
    """
    messages.append({"role": "user", "content": message})
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            max_completion_tokens=max_tokens
        )
        answer = response.choices[0].message.content
        messages.append({"role": "assistant", "content": answer})
    except Exception as e:
        print(f"Error sending message: {e}")
    return answer, messages

In [ ]:
openai_messages = []
gemini_messages = []
groq_messages = []

In [ ]:
result, openai_messages = send_message(
    client=openai_client,
    model=GPT_4_1,
    message="Hello, my name is Loic. Can you help me?",
    messages=openai_messages,
)
result

In [ ]:
result, openai_messages = send_message(
    client=openai_client,
    model=GPT_4_1,
    message="Do you remember my name?",
    messages=openai_messages
)
result

In [ ]:
openai_messages

### Mutliple Turns Conversation
Let's have fun with multiple turns conversation combining different models.

In [ ]:
def get_system_prompt(favorite_player: str, opponent_player: str) -> str:
    return f"""You are a fan of {favorite_player}, you are debating in favor of him against {opponent_player}'s fan. Be happy and enthusiastic.
    
    Be tough in debates by throwing out punchlines and using emojis, and reply with short messages.
    """

In [ ]:
openai_messages = [
    {"role": "system", "content": get_system_prompt("Lionel Messi", "Cristiano Ronaldo")},
]

gemini_messages = [
    {"role": "system", "content": get_system_prompt("Cristiano Ronaldo", "Lionel Messi")},
]

In [ ]:
max_turns = 5

openai_message = ""
gemini_message = "Hey how are you?"

print(f"Gemini (Cristiano Ronaldo): {gemini_message}")

for turn in range(max_turns):
    openai_message, openai_messages = send_message(
        client=openai_client,
        model=GPT_5_MINI,
        message=gemini_message,
        messages=openai_messages,
    )
    display(Markdown(f"**OpenAI (Lionel Messi):** {openai_message}"))

    gemini_message, gemini_messages = send_message(
        client=gemini_client,
        model=GEMINI_2_5_FLASH,
        message=openai_message,
        messages=gemini_messages,
    )
    display(Markdown(f"**Gemini (Cristiano Ronaldo):** {gemini_message}"))

In [ ]:
class DebatorItem:
    def __init__(self, client: OpenAI, model: str, favorite: str):
        self.client = client
        self.model = model
        self.favorite = favorite

In [ ]:
def debate_models(debator1: DebatorItem, debator2: DebatorItem, max_turns: int, initial_message: str):
    """
    Conduct a debate between two models with system prompts based on their favorite players.

    Args:
        debator1 (DebatorItem): The first debator item containing client, model, and favorite player.
        debator2 (DebatorItem): The second debator item containing client, model,
        max_turns: Maximum number of debate turns.
        initial_message: The initial message from the second model's fan.
    """
    system_prompt1 = get_system_prompt(debator1.favorite, debator2.favorite)
    system_prompt2 = get_system_prompt(debator2.favorite, debator1.favorite)

    messages1 = [{"role": "system", "content": system_prompt1}]
    messages2 = [{"role": "system", "content": system_prompt2}]
    
    message2 = initial_message
    display(Markdown(f"**{debator2.favorite} fan ({debator2.model}):** {initial_message}"))
    
    for turn in range(max_turns):
        message1, messages1 = send_message(debator1.client, debator1.model, message2, messages1)
        display(Markdown(f"**{debator1.favorite} fan ({debator1.model}):** {message1}"))

        message2, messages2 = send_message(debator2.client, debator2.model, message1, messages2)
        display(Markdown(f"**{debator2.favorite} fan ({debator2.model}):** {message2}"))

In [ ]:
debate_models(
    debator1=DebatorItem(
        client=groq_client,
        model=GROQ_LLAMA_3_1,
        favorite="Lionel Messi"
    ),
    debator2=DebatorItem(
        client=gemini_client,
        model=GEMINI_2_5_FLASH,
        favorite="Cristiano Ronaldo"
    ),
    max_turns=3,
    initial_message="Hey how are you?"
)

In [ ]:
# Fun debat between Gemini and Groq GPT-OSS 20B about tabs vs spaces 😂
debate_models(
    debator1=DebatorItem(
        client=groq_client,
        model=GROQ_GPT_OSS_20,
        favorite="Tabs"
    ),
    debator2=DebatorItem(
        client=gemini_client,
        model=GEMINI_2_5_FLASH,
        favorite="Spaces"
    ),
    max_turns=4,
    initial_message="Hey how are you?"
)

In [ ]:
# Political debat between Gemini and Groq Llama 3.1 about Puttinng President picture in every classroom vs not putting it 😂
debate_models(
    debator1=DebatorItem(
        client=groq_client,
        model=GROQ_LLAMA_3_1,
        favorite="Putting President picture every classroom"
    ),
    debator2=DebatorItem(
        client=gemini_client,
        model=GEMINI_2_5_FLASH,
        favorite="Not putting President picture every classroom"
    ),
    max_turns=4,
    initial_message="Salut comment ça va ?"
)

In [ ]:
# Elon Musk vs Mark Zuckerberg debate about who is the best CEO 😂
debate_models(
    debator1=DebatorItem(
        client=groq_client,
        model=GROQ_GPT_OSS_20,
        favorite="Elon Musk"
    ),
    debator2=DebatorItem(
        client=gemini_client,
        model=GEMINI_2_5_FLASH,
        favorite="Mark Zuckerberg"
    ),
    max_turns=4,
    initial_message="Hey how are you?"
)